In [ ]:
# h5py 안 될 때
!brew reinstall hdf5
!export CPATH="/opt/homebrew/include/"
!export HDF5_DIR=/opt/homebrew/
!python3 -m pip install h5py

In [210]:
import pickle
import math
from copy import deepcopy

import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [211]:
path_root = '../'
path_container = './Container/'

### Dataset 가져오기 (1) dict

In [212]:
with open(path_container + 'id_cuisine_dict.pickle', 'rb') as f:
    id_cuisine_dict = pickle.load(f)

print(id_cuisine_dict)
type(id_cuisine_dict), len(id_cuisine_dict)

{0: 'brazilian', 1: 'british', 2: 'cajun_creole', 3: 'chinese', 4: 'filipino', 5: 'french', 6: 'greek', 7: 'indian', 8: 'irish', 9: 'italian', 10: 'jamaican', 11: 'japanese', 12: 'korean', 13: 'mexican', 14: 'moroccan', 15: 'russian', 16: 'southern_us', 17: 'spanish', 18: 'thai', 19: 'vietnamese'}


(dict, 20)

In [213]:
with open(path_container + 'cuisine_id_dict.pickle', 'rb') as f:
    cuisine_id_dict = pickle.load(f)

print(cuisine_id_dict)
type(cuisine_id_dict), len(cuisine_id_dict)

{'brazilian': 0, 'british': 1, 'cajun_creole': 2, 'chinese': 3, 'filipino': 4, 'french': 5, 'greek': 6, 'indian': 7, 'irish': 8, 'italian': 9, 'jamaican': 10, 'japanese': 11, 'korean': 12, 'mexican': 13, 'moroccan': 14, 'russian': 15, 'southern_us': 16, 'spanish': 17, 'thai': 18, 'vietnamese': 19}


(dict, 20)

In [214]:
with open(path_container + 'id_ingredient_dict.pickle', 'rb') as f:
    id_ingredient_dict = pickle.load(f)

print(id_ingredient_dict[6694])
type(id_ingredient_dict), len(id_ingredient_dict)

Franks Hot Sauce


(dict, 6714)

In [215]:
with open(path_container + 'ingredient_id_dict.pickle', 'rb') as f:
    ingredient_id_dict = pickle.load(f)

print(ingredient_id_dict['lemon grass'])
print(ingredient_id_dict['lemongrass'])
print(ingredient_id_dict['bread'])
type(ingredient_id_dict), len(ingredient_id_dict)

37
561
[698, 1039, 4315]


(dict, 6694)

### Datastet 가져오기 (2) h5py

In [ ]:
"""
_train = [arr_data_train, bin_data_train, int_labels_train, bin_labels_train]
_valid_class = [arr_data_valid_class, bin_data_valid_class, int_labels_valid_class, bin_labels_valid_class]
_valid_compl = [arr_data_valid_compl, bin_data_valid_compl, int_labels_valid_compl, bin_labels_valid_compl]
_test_class = [arr_data_test_class, bin_data_test_class, None, None]
_test_compl = [arr_data_test_compl, bin_data_test_compl, None, None]
"""

In [224]:
with h5py.File(path_container + 'train', 'r') as f:
    arr_data_train = f['arr_data'][:]
    bin_data_train = f['bin_data'][:]
    int_labels_train = f['int_labels'][:]
    bin_labels_train = f['bin_labels'][:]

print(arr_data_train.shape)
print(bin_data_train.shape)
print(int_labels_train.shape)
print(bin_labels_train.shape)

(23547, 65)
(23547, 6714)
(23547,)
(23547, 20)


In [225]:
with h5py.File(path_container + 'valid_class', 'r') as f:
    arr_data_valid_class = f['arr_data'][:]
    bin_data_valid_class = f['bin_data'][:]
    int_labels_valid_class = f['int_labels'][:]
    bin_labels_valid_class = f['bin_labels'][:]

print(arr_data_valid_class.shape)
print(bin_data_valid_class.shape)
print(int_labels_valid_class.shape)
print(bin_labels_valid_class.shape)

(7848, 65)
(7848, 6714)
(7848,)
(7848, 20)


In [226]:
with h5py.File(path_container + 'valid_compl', 'r') as f:
    arr_data_valid_compl = f['arr_data'][:]
    bin_data_valid_compl = f['bin_data'][:]
    int_labels_valid_compl = f['int_labels'][:]
    bin_labels_valid_compl = f['bin_labels'][:]

print(arr_data_valid_compl.shape)
print(bin_data_valid_compl.shape)
print(int_labels_valid_compl.shape)
print(bin_labels_valid_compl.shape)

(7848, 65)
(7848, 6714)
(7848,)
(7848, 6714)


In [227]:
with h5py.File(path_container + 'test_class', 'r') as f:
    arr_data_test_class = f['arr_data'][:]
    bin_data_test_class = f['bin_data'][:]

print(arr_data_test_class.shape)
print(bin_data_test_class.shape)

(3924, 65)
(3924, 6714)


In [228]:
with h5py.File(path_container + 'test_compl', 'r') as f:
    arr_data_test_compl = f['arr_data'][:]
    bin_data_test_compl = f['bin_data'][:]

print(arr_data_test_compl.shape)
print(bin_data_test_compl.shape)

(3924, 65)
(3924, 6714)


In [233]:
# -1로 padding한 것을 6714로 갈아치우기 - for Embedding layer
pad_idx = 6714

arr_data_train[arr_data_train == -1] = pad_idx
arr_data_valid_class[arr_data_valid_class == -1] = pad_idx
arr_data_valid_compl[arr_data_valid_compl == -1] = pad_idx
arr_data_test_class[arr_data_test_class == -1] = pad_idx
arr_data_test_compl[arr_data_test_compl == -1] = pad_idx

### Ingredient feature vectors (nn.Embedding)

In [234]:
arr_data_train[0]

array([2813, 3146, 3229, 3885, 4379, 4390, 5250, 5456, 6187, 6714, 6714,
       6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
       6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
       6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
       6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
       6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714])

In [540]:
num_ingredients = pad_idx +1  # = len(id_ingredient_dict) + 1  # +1 for padding (last)
max_length = 65  # maximum num of ingredients per recipe
embedding_dim = 300


Embed = nn.Embedding(num_embeddings=num_ingredients, embedding_dim=embedding_dim, padding_idx=-1)
x = torch.LongTensor(arr_data_train[:2])

print(Embed.weight)
print(Embed.weight.size())

Parameter containing:
tensor([[-1.4323,  0.5991,  0.3715,  ..., -0.3158,  1.3486,  0.1057],
        [-0.8082, -0.2080,  1.4690,  ...,  0.5922, -1.1721,  1.8911],
        [-0.6755, -0.6658, -0.6372,  ...,  0.6404, -0.4855,  1.1124],
        ...,
        [-0.5711,  2.8740,  0.9948,  ..., -0.1664,  0.7482, -0.9478],
        [-0.2148,  1.7516,  0.1853,  ...,  0.2303,  0.9430, -0.1664],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       requires_grad=True)
torch.Size([6715, 300])


In [236]:
with torch.no_grad():
    print(Embed.weight[0])

tensor([-1.3360,  0.2779, -0.8635, -1.3298,  2.1845, -1.9543,  0.9065,  0.1224,
        -1.3027, -0.1567, -0.8056, -0.6019,  1.9613, -0.0181,  1.1118,  0.1249,
        -1.6750,  0.5685,  1.8558, -1.0680,  1.8338, -0.8267,  0.1201, -0.4727,
        -1.4298,  0.3650,  0.5593,  0.9799,  1.6882,  1.8133,  0.4485,  0.8086,
         0.1940, -0.6872,  0.4445, -0.9949,  0.7992, -1.4526,  0.6038, -0.1308,
         0.1192,  0.9464, -0.4507,  0.4774, -0.4403, -0.3338,  0.0026, -0.7624,
        -0.7807, -0.3809, -2.0264, -0.5952,  0.5740, -1.6634, -1.2131,  0.1480,
         1.3084, -1.4630, -0.1164, -0.0936,  0.6297, -0.2444, -0.6659,  1.4703,
        -0.0045, -1.1747, -0.7170,  1.6428,  0.0116,  1.1870,  0.7871, -0.0143,
        -0.4013, -2.0065,  1.4837,  0.0562,  0.2460,  0.1100,  0.6263,  1.4394,
        -0.1018, -0.2406, -1.2441,  1.2254,  0.4320, -0.5434,  0.0320, -0.8656,
        -1.0642,  1.8158, -0.1298, -0.7783, -0.9947,  0.0943,  0.5291, -0.3144,
         0.3397, -0.8688, -0.3649,  1.44

In [243]:
print(x)
with torch.no_grad():
    print(Embed(x).size())
    print(Embed(x)[0][11]) # zero padded feature matrix! (0번째 recipe는 9개라 10~65번째 feature vector는 zeros.)

tensor([[2813, 3146, 3229, 3885, 4379, 4390, 5250, 5456, 6187, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714],
        [ 392,  937, 1476, 2172, 2351, 2813, 3350, 3554, 3857, 3978, 5249, 5648,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714]])
torch.Size([2, 65, 300])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

(tensor([[[ 0,  1,  2,  3]],
 
         [[12, 13, 14, 15]]]),
 tensor([[[ 4,  5,  6,  7]],
 
         [[16, 17, 18, 19]]]),
 tensor([[[ 8,  9, 10, 11]],
 
         [[20, 21, 22, 23]]]))

### Models

Set transformer: ( https://github.com/juho-lee/set_transformer )

In [546]:
## Building blocks of Set Transformers ##
# added masks.

class MAB(nn.Module):
    def __init__(self, dim_Q, dim_K, dim_V, num_heads, ln=False):
        super(MAB, self).__init__()
        self.dim_V = dim_V
        self.num_heads = num_heads
        self.fc_q = nn.Linear(dim_Q, dim_V)
        self.fc_k = nn.Linear(dim_K, dim_V)
        self.fc_v = nn.Linear(dim_K, dim_V)
        if ln:
            self.ln0 = nn.LayerNorm(dim_V)
            self.ln1 = nn.LayerNorm(dim_V)
        self.fc_o = nn.Sequential(
            nn.Linear(dim_V, 2*dim_V),
            nn.ReLU(),
            nn.Linear(2*dim_V, dim_V))

    def forward(self, Q, K, mask=None):
        # Q (batch, q_len, d_hid)
        # K (batch, k_len, d_hid)
        # V (batch, v_len, d_hid == dim_V)
        Q = self.fc_q(Q)
        K, V = self.fc_k(K), self.fc_v(K)
        
        dim_split = self.dim_V // self.num_heads
        
        # Q_ (batch * num_heads, q_len, d_hid // num_heads)
        # K_ (batch * num_heads, k_len, d_hid // num_heads)
        # V_ (batch * num_heads, v_len, d_hid // num_heads)
        Q_ = torch.cat(Q.split(dim_split, 2), 0)
        K_ = torch.cat(K.split(dim_split, 2), 0)
        V_ = torch.cat(V.split(dim_split, 2), 0)
        
        # energy (batch * num_heads, q_len, k_len)
        energy = Q_.bmm(K_.transpose(1,2))/math.sqrt(self.dim_V)
        if mask is not None:
            energy.masked_fill_(mask, float('-inf'))
        A = torch.softmax(energy, 2)
        
        # O (batch, q_len, d_hid)
        O = torch.cat((Q_ + A.bmm(V_)).split(Q.size(0), 0), 2)
        O = O if getattr(self, 'ln0', None) is None else self.ln0(O)
        O = O + self.fc_o(O)
        O = O if getattr(self, 'ln1', None) is None else self.ln1(O)
        return O

class SAB(nn.Module):
    def __init__(self, dim_in, dim_out, num_heads, ln=False):
        super(SAB, self).__init__()
        self.mab = MAB(dim_in, dim_in, dim_out, num_heads, ln=ln)

    def forward(self, X, mask=None):
        return self.mab(X, X, mask=mask)

class ISAB(nn.Module):
    def __init__(self, dim_in, dim_out, num_heads, num_inds, ln=False):
        super(ISAB, self).__init__()
        self.I = nn.Parameter(torch.Tensor(1, num_inds, dim_out))
        nn.init.xavier_uniform_(self.I)
        self.mab0 = MAB(dim_out, dim_in, dim_out, num_heads, ln=ln)
        self.mab1 = MAB(dim_in, dim_out, dim_out, num_heads, ln=ln)

    def forward(self, X, mask=None):
        H = self.mab0(self.I.repeat(X.size(0), 1, 1), X, mask=mask)
        return self.mab1(X, H)

class PMA(nn.Module):
    def __init__(self, dim, num_heads, num_seeds, ln=False):
        super(PMA, self).__init__()
        self.S = nn.Parameter(torch.Tensor(1, num_seeds, dim))
        nn.init.xavier_uniform_(self.S)
        self.mab = MAB(dim, dim, dim, num_heads, ln=ln)
        
    def forward(self, X, mask=None):
        return self.mab(self.S.repeat(X.size(0), 1, 1), X, mask=mask)

In [547]:
class Encoder(nn.Module):
    def __init__(self, dim_input,
                 num_inds=32, dim_hidden=128, num_heads=4, num_layers=2, ln=False):
        super(Encoder, self).__init__()
        self.enc = nn.ModuleList(
            [ISAB(dim_input, dim_hidden, num_heads, num_inds, ln=ln)] +
            [ISAB(dim_hidden, dim_hidden, num_heads, num_inds, ln=ln) for _ in range(num_layers-1)]
        )
        
    def forward(self, X, mask=None):
        out = X.detach()
        for module in self.enc:
            out = module(out, mask=mask)
        return out

class Decoder(nn.Module):
    def __init__(self, num_outputs,
                 num_inds=32, dim_hidden=128, num_heads=4, ln=False):
        super(Decoder, self).__init__()
        self.dec = nn.Sequential(
                PMA(dim_hidden, num_heads, num_outputs, ln=ln),
                SAB(dim_hidden, dim_hidden, num_heads, ln=ln),
                SAB(dim_hidden, dim_hidden, num_heads, ln=ln))
        
    def forward(self, X):
        return self.dec(X)

#### Examples

In [548]:
with torch.no_grad():
    print('input size', x.size())
    print('input', x)
    feature = Embed(x)  # Size 2, 65, 300
    print('feature size', feature.size())
    print('feature', feature)

input size torch.Size([2, 65])
input tensor([[2813, 3146, 3229, 3885, 4379, 4390, 5250, 5456, 6187, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714],
        [ 392,  937, 1476, 2172, 2351, 2813, 3350, 3554, 3857, 3978, 5249, 5648,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714]])
feature size torch.Size([2, 65, 300])
feature tensor([[[-0.3718, -0.7948

In [549]:
enc_mask = (x == pad_idx).unsqueeze(1).repeat(4,1,1)
enc_mask.size(), enc_mask

(torch.Size([8, 1, 65]),
 tensor([[[False, False, False, False, False, False, False, False, False,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True]],
 
         [[False, False, False, False, False, False, False, False, False, False,
           False, False,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  T

In [550]:
x1 = deepcopy(x)
x1[0][:2] = x[0][[1,0]]
print('x1 size', x1.size())
print('x1', x1)
with torch.no_grad():
    feature1 = Embed(x1)
    print('feature1', feature1)

x1 size torch.Size([2, 65])
x1 tensor([[3146, 2813, 3229, 3885, 4379, 4390, 5250, 5456, 6187, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714],
        [ 392,  937, 1476, 2172, 2351, 2813, 3350, 3554, 3857, 3978, 5249, 5648,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714]])
feature1 tensor([[[-0.2320, -0.6004,  0.7259,  ..., -0.4812, -1.0097,  0.4035]

In [551]:
_sab = SAB(300, 128, 4)

In [552]:
with torch.no_grad():
    z = _sab(feature, mask=enc_mask)
    print('sab(feature)',z)
    print(z.size())

sab(feature) tensor([[[-1.0811,  0.8595,  0.2833,  ..., -0.7082, -0.0783, -0.3707],
         [-0.2493, -0.4630, -0.0957,  ...,  0.3101, -0.1208,  0.1451],
         [ 0.8185,  1.5390, -0.3792,  ...,  1.0239,  0.4700, -0.5495],
         ...,
         [ 0.1239,  0.0249, -0.2052,  ..., -0.0540,  0.2446, -0.3237],
         [ 0.1239,  0.0249, -0.2052,  ..., -0.0540,  0.2446, -0.3237],
         [ 0.1239,  0.0249, -0.2052,  ..., -0.0540,  0.2446, -0.3237]],

        [[-1.1312, -1.1506, -1.1365,  ..., -0.4485,  0.5151, -1.0360],
         [-0.5842,  0.6652, -0.3660,  ..., -0.0806, -0.1796,  0.8712],
         [ 0.6200, -0.4383, -0.0136,  ..., -0.1955, -0.3403,  0.6055],
         ...,
         [-0.0362,  0.1203,  0.2367,  ...,  0.0110, -0.0174,  0.0640],
         [-0.0362,  0.1203,  0.2367,  ...,  0.0110, -0.0174,  0.0640],
         [-0.0362,  0.1203,  0.2367,  ...,  0.0110, -0.0174,  0.0640]]])
torch.Size([2, 65, 128])


In [553]:
_enc = Encoder(300, dim_hidden=128) 

In [554]:
with torch.no_grad():
    code = _enc(feature,mask=enc_mask)
    print(code[0][15:])
    print(code.size())  # Size 2, 65, dim_hidden

tensor([[-0.0690,  0.0916,  0.2119,  ..., -0.0187, -0.0996,  0.0199],
        [-0.0690,  0.0916,  0.2119,  ..., -0.0187, -0.0996,  0.0199],
        [-0.0690,  0.0916,  0.2119,  ..., -0.0187, -0.0996,  0.0199],
        ...,
        [-0.0690,  0.0916,  0.2119,  ..., -0.0187, -0.0996,  0.0199],
        [-0.0690,  0.0916,  0.2119,  ..., -0.0187, -0.0996,  0.0199],
        [-0.0690,  0.0916,  0.2119,  ..., -0.0187, -0.0996,  0.0199]])
torch.Size([2, 65, 128])


In [555]:
with torch.no_grad():
    code1 = _enc(feature1, mask=enc_mask)
    print(code1)
    print(code1.size())  # Size 2, 65, dim_hidden  # permutation equivariant! (observe first 2 rows)

tensor([[[-0.1489,  0.3277, -0.1951,  ..., -0.0156, -0.1569,  0.4657],
         [-0.5638,  0.8315, -0.1925,  ..., -0.3830,  0.2357, -0.1620],
         [-0.3019,  0.5831,  0.8459,  ..., -0.1022, -0.3173, -0.2990],
         ...,
         [-0.0690,  0.0916,  0.2119,  ..., -0.0187, -0.0996,  0.0199],
         [-0.0690,  0.0916,  0.2119,  ..., -0.0187, -0.0996,  0.0199],
         [-0.0690,  0.0916,  0.2119,  ..., -0.0187, -0.0996,  0.0199]],

        [[-0.2554,  0.3312, -0.0621,  ..., -0.3660,  0.0441, -0.1231],
         [-0.4615, -0.3753, -0.6948,  ...,  0.2721,  0.3118,  0.4291],
         [-0.2623,  0.6359,  0.4309,  ..., -0.5692, -0.7051,  0.1332],
         ...,
         [-0.1413,  0.0849, -0.0211,  ..., -0.1963, -0.1153,  0.0526],
         [-0.1413,  0.0849, -0.0211,  ..., -0.1963, -0.1153,  0.0526],
         [-0.1413,  0.0849, -0.0211,  ..., -0.1963, -0.1153,  0.0526]]])
torch.Size([2, 65, 128])


In [556]:
_dec = Decoder(1, dim_hidden=128)

In [557]:
with torch.no_grad():
    out = _dec(code)
    print(out)
    print(out.size())  # Size 2, num_output, dim_hidden. FF 적용 직전 vector.

tensor([[[-0.1978,  0.0129,  0.0625,  0.0120,  0.0346,  0.1257, -0.0215,
          -0.1831,  0.0467,  0.0193,  0.0717, -0.1412,  0.0530, -0.0501,
          -0.1162,  0.0165,  0.1132,  0.0610, -0.1639,  0.0273,  0.0963,
          -0.0051, -0.1811,  0.0597, -0.0864,  0.3383, -0.3138,  0.0682,
          -0.0637,  0.0596, -0.0268, -0.1935,  0.5056,  0.3642, -0.0314,
           0.2691, -0.1282, -0.3630, -0.0420, -0.0471,  0.1804,  0.1003,
          -0.1876, -0.2552, -0.1873,  0.1398,  0.0695, -0.0252, -0.0293,
          -0.0218,  0.0517,  0.0418, -0.0733,  0.1663,  0.0441,  0.1436,
           0.0106,  0.0543, -0.2302, -0.1391,  0.0185,  0.2868, -0.0059,
           0.0476,  0.0071, -0.1018,  0.1085,  0.0848, -0.2863, -0.1488,
          -0.0353,  0.0657, -0.1502, -0.0441, -0.0057, -0.0644, -0.0555,
          -0.2947,  0.1081, -0.2980, -0.1075,  0.3180,  0.0098,  0.1202,
           0.1133, -0.0935,  0.2495,  0.0615,  0.0846, -0.1701, -0.0205,
           0.0753, -0.0851, -0.0896,  0.2262, -0.04

In [558]:
# permutation invariance

with torch.no_grad():
    x2 = deepcopy(x)
    """3146, 2813, 3229, 3885, 4379, 4390, 5250, 5456, 6187"""
    shuf_ind = torch.randperm(x2.size(1))
    x2[0] = x2[0][shuf_ind]  # ramdom permuted!
    print(x2[0])
    feature2 = Embed(x2)
    enc_mask2 = deepcopy(enc_mask)
    enc_mask2[0] = enc_mask[0][0][shuf_ind]
    code2 = _enc(feature2, mask=enc_mask2)
    out2 = _dec(code2)
    print(out2.size())
    print(out2[0].isclose(out[0]))
    print(out2[0] - out[0]) # almost the same.

tensor([6714, 6714, 3229, 6714, 6714, 6714, 6714, 6714, 6714, 3146, 6714, 2813,
        6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
        6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
        6714, 5456, 6714, 6187, 6714, 6714, 6714, 4379, 6714, 6714, 6714, 6714,
        6714, 6714, 4390, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
        6714, 5250, 6714, 3885, 6714])
torch.Size([2, 1, 128])
tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False

### Unified Model: Classification + Completion (CCNet)

In [574]:
_a = torch.LongTensor([1,2,3,1,2,3])
torch.Tensor(list(set([1,2,3])))

tensor([1., 2., 3.])

In [589]:
class CCNet(nn.Module):
    def __init__(self, dim_input=256, dim_output=20, num_items=6714+1, 
                  num_inds=32, dim_hidden=128, num_heads=4, num_outputs=1+1,  # class 1 + compl 1
                  num_enc_layers=4, num_dec_layers=2, ln=False):
        super(CCNet, self).__init__()
        
        self.num_heads = num_heads
        self.padding_idx = num_items-1
        self.embedding =  nn.Embedding(num_embeddings=num_items, embedding_dim=dim_input, padding_idx=-1)
        self.encoder = nn.ModuleList(
            [ISAB(dim_input, dim_hidden, num_heads, num_inds, ln=ln)] +
            [ISAB(dim_hidden, dim_hidden, num_heads, num_inds, ln=ln) for _ in range(num_enc_layers-1)])
        self.pooling = PMA(dim_hidden, num_heads, num_outputs, ln=ln)
        self.decoder1 = nn.Sequential(
            *[SAB(dim_hidden, dim_hidden, num_heads, ln=ln) for _ in range(num_dec_layers)])
        self.ff1 = nn.Sequential(
            nn.Linear(dim_hidden, dim_hidden),
            nn.ReLU(),
            nn.Linear(dim_hidden, dim_output))
        self.decoder2 = nn.ModuleList(
            [MAB(dim_hidden, dim_input, dim_hidden, num_heads, ln=ln) for _ in range(2)])
        self.ff2 = nn.Linear(dim_hidden, num_items-1)
    
    def generate_mask(self, x):
        return (torch.LongTensor(x) == self.padding_idx).repeat(self.num_heads,1).unsqueeze(1)

    
    def forward(self, x, bin_x):
        # x(=recipes): (batch, max_num_ingredient=65)
        # bin_x : (batch, num_items-1=6714)
        
        feature = self.embedding(torch.LongTensor(x))
        # feature: (batch, max_num_ingredient=65, dim_input=256)
        # cf. embedding.weight: (num_items=6715, dim_input=256)
        
        mask = self.generate_mask(x)
        # mask: (batch*num_heads, 1, max_num_ingredient=65)
        
        code = feature.clone()
        for module in self.encoder:
            code = module(code, mask=mask)
        # code: (batch, max_num_ingredient=65, dim_hidden=128) : permutation-equivariant.
        
        pooled = self.pooling(code, mask=mask)
        # pooled: (batch, num_outputs=2, dim_hidden=128) : permutation-invariant.
        
        signals = self.decoder1(pooled)
        # no mask; signals: (batch, num_outputs=2, dim_hidden=128) : permutation-invariant.
        
        # split two signals: for classification & completion.
        signal_classification = signals[:][0].squeeze()  # (batch, dim_hidden=128)
        signal_completion = signals.clone()[:][1]        # (batch, 1, dim_hidden=128)
        
        # Classification:
        logit_classification = self.ff1(signal_classification) # (batch, dim_output)
        
        # Completion:
        used_ingred_mask = self.generate_mask(bin_x)
        # used_ingred_mask: (batch*num_heads, 1, num_items-1=6714)
        embedding_weight = self.embedding.weight[:-1].unsqueeze(0).repeat(feature.size(0),1,1)
        # embedding_weight: (batch, num_items=6715, dim_input=256)
        for module in self.decoder2:
            signal_completion = module(signal_completion, embedding_weight, mask=used_ingred_mask)
        logit_completion = self.ff2(signal_completion.squeeze()) # (batch, num_items-1=6714)
        logit_completion[used_ingred_mask.squeeze()] = float('-inf')
        
        return logit_classification, logit_completion

In [590]:
_model = CCNet()

In [591]:
_model(arr_data_train[:2], bin_data_train[:2])

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)